# DeepLabV3+ Transfer Learning Training Demo

DeepLabV3+ with interchangeable backbone transfer learning training demo
Net pretrained with COCO val2017, then fine-tuned with Yamaha-CMU Off-Road Dataset

---

Author: **Nate Haddad** nhaddad2112[at]gmail[dot]com

In [58]:
import os.path as op
import sys
import os
import torch
from torchvision import models
from torchvision.models.segmentation.deeplabv3 import DeepLabHead
import numpy as np
from PIL import Image
import yaml

sys.path.append('H:/Courses/DL_Projects/IS_project/semantic-segmentation/')
from utils import get_dataloader, Trainer, display_example_pair
from models import DeepLabWrapper

In [59]:
torch.cuda.is_available()
# code to see  if GPU is available, and if not run on CPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

def check_gpu():
    if torch.cuda.is_available():
        print("Num GPUs Available:", torch.cuda.device_count())
        for i in range(torch.cuda.device_count()):
            print(f"GPU {i}:", torch.cuda.get_device_name(i))
        print("Using device:", device)
    else:
        print("No GPU device found.")

check_gpu()

Num GPUs Available: 1
GPU 0: NVIDIA GeForce RTX 3060 Laptop GPU
Using device: cuda:0


In [48]:
np.random.seed(42)

Set up all of our path variables and hyperparamters needed to train the model. Adjust accordingly to your needs

In [49]:
with open('H:/Courses/DL_Projects/IS_project/semantic-segmentation/config/config.yaml', 'r') as f:
    config = yaml.safe_load(f)

Set up the `data` and `model` directories

In [50]:
os.makedirs('H:/Courses/DL_Projects/IS_project/semantic-segmentation/data', exist_ok = True)
os.makedirs('H:/Courses/DL_Projects/IS_project/semantic-segmentation/runs', exist_ok = True)

---

**IMPORTANT:** we require that you now download the Yamaha CMU Off-Road Dataset from the following url before proceding. The directory and files should be unzipped and placed in the `data` directory of this repository, created above.

**Link to download dataset:** https://theairlab.org/yamaha-offroad-dataset/

---

Load an example image from the training set

In [51]:
example_image = Image.open(op.join('H:/Courses/DL_Projects/IS_project/semantic-segmentation/', config['DATA_PATH'], 'train/iid000008/rgb.jpg'))
example_mask = Image.open(op.join('H:/Courses/DL_Projects/IS_project/semantic-segmentation/', config['DATA_PATH'], 'train/iid000008/labels.png'))
image_display = np.array(example_image)
mask_display = np.array(example_mask.convert('RGB'))
#display_example_pair(image_display, mask_display)

Load data into dataloaders

In [52]:
dataloaders = get_dataloader(op.join('H:/Courses/DL_Projects/IS_project/semantic-segmentation/', config['DATA_PATH']),
                             batch_size=config['BATCH_SIZE'],
                             resize_shape=(config['IMG_HEIGHT'],
                             config['IMG_WIDTH']))

### Model Selection

Create the model. Note that the model is pretrained on COCO val2017. Replace the output layer with 8 mask channels and the correct number of nodes for the last layer

In [53]:
model = DeepLabWrapper(backbone=config['BACKBONE'],
                       num_mask_channels=config['NUM_MASK_CHANNELS'],
                       pretrained=True,
                       progress=True,
                       aux_loss=True)

Freeze earlier layers

In [42]:
# freeze_layers(model, 0, 14)

Select the optimizer and loss function

In [54]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters, lr=float(config['LEARNING_RATE']))

Train the model

In [55]:
trainer = Trainer(model, dataloaders, criterion, optimizer, num_epochs=config['NUM_EPOCHS'], is_inception=config['IS_INCEPTION']);

In [56]:
trainer.train()

Epoch 1/30
----------


100%|██████████| 465/465 [01:46<00:00,  4.36it/s]


train Loss: 2.1595 mIoU: 0.8420


100%|██████████| 72/72 [00:08<00:00,  8.66it/s]


valid Loss: 1.4791 mIoU: 0.8242

Epoch 2/30
----------


...

100%|██████████| 465/465 [01:37<00:00,  4.76it/s]


train Loss: 0.4675 mIoU: 0.9768


100%|██████████| 72/72 [00:07<00:00, 10.14it/s]

valid Loss: 1.3802 mIoU: 0.8181

Training complete in 52m 16s
Best val mean IoU: 0.837854


(DeepLabWrapper(
   (model): DeepLabV3(
     (backbone): IntermediateLayerGetter(
       (0): ConvNormActivation(
         (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
         (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
         (2): Hardswish()
       )
       (1): InvertedResidual(
         (block): Sequential(
           (0): ConvNormActivation(
             (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
             (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
             (2): ReLU(inplace=True)
           )
           (1): ConvNormActivation(
             (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
             (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
           )
         )
       )
       (2): InvertedResidual(
         (block): Sequential(
     

---
Save the entire model

In [57]:
model.save_model(op.join('H:/Courses/DL_Projects/IS_project/semantic-segmentation/', config['SAVE_MODEL_PATH']))